In [1]:
from os import getcwd, path, environ
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

BASE_PATH = getcwd()
sys.path.append(BASE_PATH)

environ['NUM_WORKERS'] = '0'
# environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
data_imdb = pd.read_json('data/imdb/train.json')
data_binary_sst = pd.read_csv('data/binary_sst/train.csv')
# data_semeval = pd.read_csv('data/semeval/train.csv')
data = pd.concat([data_imdb, data_binary_sst], ignore_index=True)

val_data_imdb = pd.read_json('data/imdb/test.json')
val_data_binary_sst = pd.read_csv('data/binary_sst/test.csv')
# data_semeval = pd.read_csv('data/semeval/train.csv')
val_data = pd.concat([val_data_imdb, val_data_binary_sst], ignore_index=True)

data.head()

/home/luungoc2005/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.
/home/luungoc2005/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,id,label,sentence
0,57447.0,1,This was on at 2 or so In the morning one Satu...
1,42373.0,0,JUDAAI was a bold film by Raj Kanwar at it's t...
2,1158910.0,1,"Man, I loved this movie! This really takes me ..."
3,8571.0,0,Many King fans hate this because it departed f...
4,120003.0,0,The only possible way to enjoy this flick is t...


In [3]:
data.loc[data['label'] == 0, 'label'] = 'negative'
data.loc[data['label'] == 1, 'label'] = 'positive'

val_data.loc[val_data['label'] == 0, 'label'] = 'negative'
val_data.loc[val_data['label'] == 1, 'label'] = 'positive'

data.head()

,id,label,sentence
0,57447.0,positive,This was on at 2 or so In the morning one Satu...
1,42373.0,negative,JUDAAI was a bold film by Raj Kanwar at it's t...
2,1158910.0,positive,"Man, I loved this movie! This really takes me ..."
3,8571.0,negative,Many King fans hate this because it departed f...
4,120003.0,negative,The only possible way to enjoy this flick is t...


In [4]:
data['sentence'] = data['sentence'].str.replace('\n', ' ').replace('<br />', ' ').replace('  ', ' ')

In [5]:
from sent_to_vec.masked_lm.bert_model import BertLMWrapper
from sent_to_vec.masked_lm.train import LanguageModelLearner
from sent_to_vec.masked_lm.data import WikiTextDataset

from common.modules import BertAdam
from common.callbacks import PrintLoggerCallback, EarlyStoppingCallback, ModelCheckpointCallback, TensorboardCallback, ReduceLROnPlateau

Default language for this instance: en


In [6]:
model = BertLMWrapper(from_fp='bert_en_base.bin')
model.init_model()

dataset = WikiTextDataset()
dataset.initialize(model, data_texts=list(data['sentence']))

Featurizer previously fitted, continuing
Found 1732377 tokens


In [7]:
BATCH_SIZE = 16
n_epochs=5

learner = LanguageModelLearner(model,
    optimizer_fn=BertAdam,
    optimizer_kwargs={
        'lr': 2e-5,
        't_total': n_epochs * (len(dataset) // BATCH_SIZE),
        'warmup': 0.04
    })

Training in BERT mode


In [8]:
import torch
learner.fit(
    training_data=dataset,
    batch_size=BATCH_SIZE,
    epochs=n_epochs,
    callbacks=[
        PrintLoggerCallback(log_every=1, metrics=['loss']),
    ],
    gradient_accumulation_steps=10,
    clip_grad=1.0,
)


Gradient accumulation is supported by this class
Number of tokens 36000
BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36000, 576, padding_idx=0)
      (position_embeddings): Embedding(104, 576, padding_idx=0)
      (token_type_embeddings): Embedding(2, 576, padding_idx=0)
      (LayerNorm): FusedLayerNorm(torch.Size([576]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=576, out_features=576, bias=True)
              (key): Linear(in_features=576, out_features=576, bias=True)
              (value): Linear(in_features=576, out_features=576, bias=True)
              (dropout): Dropout(p=0.15)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=576, out_features=576

In [9]:
# model.save('bert_vi_sentiment_lm.bin')

In [10]:
from text_classification.with_pretrained.model import LMClassifierWrapper
from text_classification.with_pretrained.train import LMClassifierLearner

In [11]:
n_epochs=30
classifier = LMClassifierWrapper(encoder=model)
classifier_learner = LMClassifierLearner(
    classifier,
    optimizer_fn=BertAdam,
    optimizer_kwargs={
        'lr': 2e-5,
        't_total': n_epochs * (len(dataset) // BATCH_SIZE),
        'warmup': 0.04
    }
)

In [12]:
# classifier.model.encoder

In [13]:
classifier_learner.fit(
    training_data=(data['sentence'], data['label']),
    validation_data=(val_data['sentence'], val_data['label']),
    batch_size=BATCH_SIZE,
    epochs=n_epochs, 
    callbacks=[
        PrintLoggerCallback(log_every=1),
        ModelCheckpointCallback()
    ]
)

{'num_words': 36000, 'hidden_size': 576, 'num_hidden_layers': 6, 'num_attention_heads': 12, 'intermediate_size': 1200, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.15, 'attention_probs_dropout_prob': 0.15, 'max_position_embeddings': 104, 'featurizer_seq_len': 104, 'type_vocab_size': 2, 'initializer_range': 0.025, 'use_adasoft': True, 'append_sos_eos': True, 'featurizer_reserved_tokens': ['<START>', '<STOP>', '<UNK>', '<MASK>'], 'tokenize_fn': <function word_tokenize at 0x7f8519bf7c80>, 'input_shape': (None,), 'adasoft_cutoffs': [2000, 4000, 10000]}


/media/luungoc2005/Data/Projects/botbot-nlp/common/wrappers.py:582: UserWarning: Error orcurred in multiprocessing.set_start_method
  warnings.warn('Error orcurred in multiprocessing.set_start_method')


3m 29s (- 101m 5s) (1 3%) - loss: 0.6223 - accuracy: 0.6769
TEST:  - val_accuracy: 0.4951
Model Checkpoint: Saving checkpoint: _ 3m 29s _epoch 1_1995 loss_0_6223 accuracy_0_6769.bin
7m 6s (- 99m 32s) (2 6%) - loss: 0.4655 - accuracy: 0.7764
TEST:  - val_accuracy: 0.5086
Model Checkpoint: Saving checkpoint: _ 7m 6s _epoch 2_1995 loss_0_4655 accuracy_0_7764.bin
10m 31s (- 94m 43s) (3 10%) - loss: 0.3966 - accuracy: 0.8174
TEST:  - val_accuracy: 0.5121
Model Checkpoint: Saving checkpoint: _ 10m 31s _epoch 3_1995 loss_0_3966 accuracy_0_8174.bin
13m 35s (- 88m 18s) (4 13%) - loss: 0.3348 - accuracy: 0.8537
TEST:  - val_accuracy: 0.5062
Model Checkpoint: Saving checkpoint: _ 13m 35s _epoch 4_1995 loss_0_3348 accuracy_0_8537.bin
16m 39s (- 83m 15s) (5 16%) - loss: 0.2783 - accuracy: 0.8817
TEST:  - val_accuracy: 0.5069
Model Checkpoint: Saving checkpoint: _ 16m 39s _epoch 5_1995 loss_0_2783 accuracy_0_8817.bin
19m 43s (- 78m 54s) (6 20%) - loss: 0.2286 - accuracy: 0.9052
TEST:  - val_accuracy

In [14]:
test_data_imdb = pd.read_json('data/imdb/test.json')
test_data_binary_sst = pd.read_csv('data/binary_sst/test.csv')
# data_semeval = pd.read_csv('data/semeval/train.csv')
test_data = pd.concat([test_data_imdb, test_data_binary_sst], ignore_index=True)

/home/luungoc2005/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [15]:
test_data.head()

,id,label,sentence
0,115393.0,0,"ExCUSE me, but my tongue was TOO in my cheek w..."
1,114277.0,1,Mud and Sand is one of Stan Laurel's spoofs of...
2,3449.0,1,I saw this film at the 2002 Toronto Internatio...
3,20288.0,1,WARNING - POSSIBLE SPOILERS!<br /><br />'Rock ...
4,4624.0,0,The filmmakers apparently had enough money to ...


In [16]:
import torch
test_item = [test_data['sentence'][5]]
classifier(test_item, return_logits=True)[0]

tensor([[2.0647e-03, 9.9794e-01, 1.8923e-09]], device='cuda:0')

In [17]:
classifier.save('bert_en_sentiment.bin')

In [18]:
raw_model = classifier.model

In [19]:
classifier.label_encoder.classes_.tolist()

['negative', 'positive']

In [20]:
classifier(test_item)

[[{'intent': 'positive', 'confidence': 0.99793541431427},
  {'intent': 'negative', 'confidence': 0.002064651809632778}]]

In [21]:
# from common.torch_utils import to_gpu

# _, seq_tokens = raw_model.encoder(
#     to_gpu(classifier.featurizer.transform(test_item))
# )
# sequence_output = raw_model.rnn(seq_tokens)[0]
# print(sequence_output)

# output, idxs = torch.max(sequence_output, 0)
# print(idxs)
# idxs = idxs.data.cpu().numpy()

# sent = classifier.featurizer.transform(test_item)
# raw_sent = classifier.featurizer.inverse_transform(sent)
# print(raw_sent)

# import matplotlib.pyplot as plt

# argmaxs = [np.sum((idxs == k)) for k in range(len(sent[0]))]
# # argmaxs[0] = 1e-8
# print(argmaxs)
# x = range(len(sent[0]))
# y = [100.0 * n / np.sum(argmaxs) for n in argmaxs]
# print(y)

# plt.xticks(x, raw_sent[0], rotation=45)
# plt.bar(x, y)
# plt.ylabel('%')
# plt.title('Visualisation of words importance')
# plt.show()

In [22]:
loaded_model = LMClassifierWrapper(from_fp='bert_en_sentiment.bin')
loaded_model.init_model()

print(loaded_model(test_item))

{'num_words': 36000, 'hidden_size': 576, 'num_hidden_layers': 6, 'num_attention_heads': 12, 'intermediate_size': 1200, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.15, 'attention_probs_dropout_prob': 0.15, 'max_position_embeddings': 104, 'featurizer_seq_len': 104, 'type_vocab_size': 2, 'initializer_range': 0.025, 'use_adasoft': True, 'append_sos_eos': True, 'featurizer_reserved_tokens': ['<START>', '<STOP>', '<UNK>', '<MASK>'], 'tokenize_fn': <function word_tokenize at 0x7f8519bf7c80>, 'input_shape': (None,), 'adasoft_cutoffs': [2000, 4000, 10000]}
[[{'intent': 'positive', 'confidence': 0.99793541431427}, {'intent': 'negative', 'confidence': 0.002064651809632778}]]


In [23]:
classifier(test_item, return_logits=True)[0]

tensor([[2.0647e-03, 9.9794e-01, 1.8923e-09]], device='cuda:0')

In [24]:
print(loaded_model.__getstate__()['state_dict'])

OrderedDict([('encoder.bert.embeddings.word_embeddings.weight', tensor([[-0.0104,  0.0358,  0.3164,  ..., -0.0229,  0.0094,  0.0340],
        [-0.0314,  0.0202,  0.1967,  ..., -0.0490,  0.0160,  0.0268],
        [-0.0032, -0.0205,  0.1533,  ..., -0.0065,  0.0032,  0.0137],
        ...,
        [ 0.0065,  0.0319, -0.0164,  ..., -0.0071, -0.0037, -0.0153],
        [ 0.0005, -0.0079, -0.0025,  ..., -0.0315,  0.0131,  0.0068],
        [ 0.0301, -0.0161,  0.0007,  ...,  0.0288,  0.0162, -0.0064]])), ('encoder.bert.embeddings.position_embeddings.weight', tensor([[-0.0086,  0.0177, -0.0065,  ...,  0.0058,  0.0254,  0.0090],
        [-0.0042, -0.0197, -0.0356,  ..., -0.0568, -0.0090, -0.0168],
        [-0.0025, -0.0180, -0.0257,  ..., -0.0250,  0.0334, -0.0053],
        ...,
        [-0.0927,  0.0053, -0.0284,  ..., -0.0339, -0.0140,  0.0185],
        [ 0.0019,  0.0028, -0.0563,  ..., -0.0569, -0.0402,  0.0200],
        [-0.0433, -0.0250, -0.0273,  ..., -0.0343,  0.0037, -0.0157]])), ('encoder

In [25]:
print(classifier.__getstate__()['state_dict'])

OrderedDict([('encoder.bert.embeddings.word_embeddings.weight', tensor([[-0.0104,  0.0358,  0.3164,  ..., -0.0229,  0.0094,  0.0340],
        [-0.0314,  0.0202,  0.1967,  ..., -0.0490,  0.0160,  0.0268],
        [-0.0032, -0.0205,  0.1533,  ..., -0.0065,  0.0032,  0.0137],
        ...,
        [ 0.0065,  0.0319, -0.0164,  ..., -0.0071, -0.0037, -0.0153],
        [ 0.0005, -0.0079, -0.0025,  ..., -0.0315,  0.0131,  0.0068],
        [ 0.0301, -0.0161,  0.0007,  ...,  0.0288,  0.0162, -0.0064]])), ('encoder.bert.embeddings.position_embeddings.weight', tensor([[-0.0086,  0.0177, -0.0065,  ...,  0.0058,  0.0254,  0.0090],
        [-0.0042, -0.0197, -0.0356,  ..., -0.0568, -0.0090, -0.0168],
        [-0.0025, -0.0180, -0.0257,  ..., -0.0250,  0.0334, -0.0053],
        ...,
        [-0.0927,  0.0053, -0.0284,  ..., -0.0339, -0.0140,  0.0185],
        [ 0.0019,  0.0028, -0.0563,  ..., -0.0569, -0.0402,  0.0200],
        [-0.0433, -0.0250, -0.0273,  ..., -0.0343,  0.0037, -0.0157]])), ('encoder

In [26]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')


In [27]:
compare_models(loaded_model.model, classifier.model)

Models match perfectly! :)


In [28]:
loaded_model.featurizer.transform(test_item)

tensor([[   1, 3522,   81,   45, 1571, 4696, 1180, 3956,    6,   31,    3,   66,
            5,  106,    3, 1071,   18,   32, 3701,    7,   79, 3751, 1632,  128,
         4244,  356,  111,   31,  373,    3, 1542,    5,  534, 1071,    7,  138,
          696,   29,   32,  375,   19, 1566,    7,    3,    3,  123,    3,    3,
            3,  123,    3, 1180, 1677,  618,   19,    5, 1566,   10,    3,   34,
           39, 1395,   58, 1125,    9,    5,    3,   18,  739, 2781,   18,   32,
         1662,  376,  113,    3,    7,   79, 2594,  108,   11,    3,   18,   32,
         3701,  328,    5,   96,   81,   31,  448, 1632, 4244,  356,   29,   94,
           20,    3,   48,    3,   28,   52,   11,    3]])

In [29]:
X_train = classifier.featurizer.transform(test_item).cuda()

In [30]:
raw_loaded_model = loaded_model.model
raw_classifier = classifier.model

In [31]:
raw_loaded_model.eval()
raw_classifier.eval()
raw_loaded_model.encoder = raw_classifier.encoder
raw_loaded_model(X_train)[0] - raw_classifier(X_train)[0]

tensor([[0., 0., 0.]], device='cuda:0', grad_fn=<SubBackward0>)

In [32]:
from common.torch_utils import cauchy
raw_loaded_model.eval()
raw_classifier.eval()

encoded = raw_loaded_model.encoder(X_train)[1]
rnn = raw_loaded_model.rnn(encoded)[0]
max_pool = torch.max(rnn, 1)[0]
pooled = max_pool
prev_pooled = max_pool
# print(prev_pooled - pooled)
# pooled = raw_loaded_model.pooler(max_pool)
for ix, layer in enumerate(raw_loaded_model.pooler):
    pooled = layer(pooled)
    prev_pooled = raw_classifier.pooler[ix](prev_pooled)
#     print(ix)
#     print(prev_pooled - pooled)
#     print(layer.weight - raw_classifier.pooler[ix].weight)
#     print(layer.bias - raw_classifier.pooler[ix].bias)
#     print(ix)
#     print(len(raw_loaded_model.pooler))
#     print(pooled - prev_pooled)
    if ix < len(raw_loaded_model.pooler) - 1:
        pooled = torch.nn.functional.relu(pooled)
        prev_pooled = torch.nn.functional.relu(prev_pooled)
#         print(prev_pooled - pooled)
#         print(pooled)
    else:
        pooled = cauchy(pooled)
        prev_pooled = cauchy(prev_pooled)
#         print(prev_pooled - pooled)
#         print(pooled)

# print(pooled - prev_pooled)
print(raw_loaded_model.classifier.bias)
print(raw_classifier.classifier.bias)
with torch.no_grad():
    print(raw_loaded_model.classifier.weight - raw_classifier.classifier.weight)
    print(raw_loaded_model.classifier(prev_pooled))
    print(raw_classifier.classifier(prev_pooled))

AttributeError: 'LMClassifier' object has no attribute 'rnn'

In [ ]:
print(max_pool)

In [ ]:
print(loaded_model.model.encoder == raw_loaded_model.encoder)
print(loaded_model(test_item))